In [1]:
# Client Setup
import boto3

#client = boto3.client("bedrock-runtime", region_name="us-east-1")
session = boto3.Session(profile_name="bedrock-dev")
client = session.client("bedrock-runtime", region_name="us-west-2")
# Claude model ID — must be correct and available in the region
model_id = "us.anthropic.claude-3-sonnet-20240229-v1:0"

# Note: You might have to request access to this model on the AWS Bedrock console
#embedding_model_id = "amazon.titan-embed-text-v2:0"
embedding_model_id = "amazon.titan-embed-text-v1"

In [2]:
# Helper functions


def add_user_message(messages, content):
    if isinstance(content, str):
        user_message = {"role": "user", "content": [{"text": content}]}
    else:
        user_message = {"role": "user", "content": content}
    messages.append(user_message)


def add_assistant_message(messages, content):
    if isinstance(content, str):
        assistant_message = {
            "role": "assistant",
            "content": [{"text": content}],
        }
    else:
        assistant_message = {"role": "assistant", "content": content}

    messages.append(assistant_message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    tool_choice="auto",
    text_editor=None,
    thinking=False,
    thinking_budget=1024,
):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    tool_choices = {
        "auto": {"auto": {}},
        "any": {"any": {}},
    }
    if tools or text_editor:
        choice = tool_choices.get(tool_choice, {"tool": {"name": tool_choice}})
        params["toolConfig"] = {"tools": tools, "toolChoice": choice}

    additional_model_fields = {}
    if text_editor:
        additional_model_fields["tools"] = [
            {
                "type": text_editor,
                "name": "str_replace_editor",
            }
        ]

    if thinking:
        additional_model_fields["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    params["additionalModelRequestFields"] = additional_model_fields
    response = client.converse(**params)
    parts = response["output"]["message"]["content"]

    return {
        "parts": parts,
        "stop_reason": response["stopReason"],
        "text": "\n".join([p["text"] for p in parts if "text" in p]),
    }


In [3]:
# Fire risk assessment prompt
prompt = """
Analyze the attached satellite image of a property with these specific steps:

1. Residence identification. Locate the primary residence on the property by looking for:
   - The largest roofed structure 
   - Typical residential features (driveway connection, regular geometry)
   - Distinction from other structures (garages, sheds, pools)
   Describe the residence's location relative to property boundaries and other features.

2. Tree density analysis. Examine the overall distribution and density of trees on the property:
   - Identify areas where trees are closely packed together
   - Assess the spacing between tree canopies (isolated, clustered, or continuous)
   - Evaluate whether fire could easily spread from tree to tree based on proximity
   - Note any particularly dense wooded areas in relation to the main residence

3. Access assessment. Evaluate emergency vehicle access to the primary residence:
   - Identify clear paths, roads, or driveways leading to or from the main structure
   - Assess whether access routes are wide enough for emergency vehicles
   - Note any potential obstructions (gates, narrow passages, overhanging vegetation)
   - Consider if multiple access points exist or if there is only a single entry/exit route

4. Roof overhang analysis. Examine tree branches in relation to the main residence:
   - Identify any trees whose canopy extends directly over any portion of the roof
   - Estimate the percentage of roof covered by overhanging branches (0-25%, 25-50%, 50-75%, 75-100%)
   - Note particularly dense areas of overhang or contact points between branches and structure
   - Assess proximity of branches to chimneys, vents, or other roof openings if visible

5. Fire risk rating. Based on your analysis, assign a Fire Risk Rating from 1-4:
   - Rating 1 (Low Risk): Well-spaced trees with no canopy continuity, clear access routes, no branches overhanging the roof
   - Rating 2 (Moderate Risk): Some clustered trees but with breaks in continuity, adequate access routes, minimal roof overhang (<25%)
   - Rating 3 (High Risk): Densely packed trees with connected canopies, limited or narrow access routes, significant roof overhang (25-50%)
   - Rating 4 (Severe Risk): Continuous tree canopy coverage across property, restricted or obstructed access routes, extensive roof overhang (>50%)

For each item above (1-5), write one sentence summarizing your findings, with your final response being the numeric Fire Risk Rating (1-4) with a brief justification.
"""

In [4]:
with open("./images/prop3.png", "rb") as f:
    image_bytes = f.read()

messages = []

add_user_message(
    messages,
    [
        {"image": {"format": "png", "source": {"bytes": image_bytes}}},
        {"text": prompt},
    ],
)

response = chat(messages)

response["text"]

"1. The primary residence appears to be the large rectangular structure located near the center of the property, featuring a driveway connection and distinct roof geometry compared to the smaller adjacent structures likely used for storage or other auxiliary purposes.\n\n2. The property has a moderate density of trees, with areas of clustered canopies as well as some more isolated tree spacing, suggesting the potential for fire to spread from tree to tree in certain sections, particularly the wooded area situated relatively close to the main residence.\n\n3. Emergency vehicle access seems adequate, with a visible driveway leading directly to the primary structure, though the width and any potential obstructions along the path cannot be definitively assessed from this aerial perspective alone.\n\n4. Several trees have branches extending over portions of the residence's roof, with an estimated 25-50% of the roof area covered by overhanging canopy based on the visible density and proximit